In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

: 

In [2]:
data = pd.read_csv("sample_data/prepareddata2.csv")

print(data)

        Unnamed: 0  url_length  domain_length  has_https  has_ipv4  \
0                0        22.0           14.0        1.0       0.0   
1                1        23.0           15.0        1.0       0.0   
2                2        24.0           16.0        1.0       0.0   
3                3        21.0           13.0        1.0       0.0   
4                4        25.0           17.0        1.0       0.0   
...            ...         ...            ...        ...       ...   
450171      450171        43.0           11.0        0.0       0.0   
450172      450172       159.0           13.0        0.0       0.0   
450173      450173       147.0           13.0        0.0       0.0   
450174      450174        22.0           14.0        0.0       0.0   
450175      450175       143.0           18.0        0.0       0.0   

        parameter_count  letter_count  nb_count  non_alpha_count  ques  \
0                   1.0          17.0       0.0              5.0   0.0   
1          

In [3]:
data = data.dropna(subset=["has_ipv4"])
print(data.isnull().sum())

Unnamed: 0         0
url_length         0
domain_length      0
has_https          0
has_ipv4           0
parameter_count    0
letter_count       0
nb_count           0
non_alpha_count    0
ques               0
slashes            0
dashes             0
underscores        0
dots               0
has_php            0
has_html           0
result             0
dtype: int64


In [4]:
x = data[["url_length", "domain_length", "has_https", "parameter_count", "letter_count",
         "nb_count", "non_alpha_count", "ques", "slashes", "dashes", "underscores", "dots",
         "has_php", "has_html"]]



**Part 1: K-Nearest Neighbor Model**

In [5]:
y = data["result"]

training_x, testing_x, training_y, testing_y = train_test_split(x, y, test_size=0.2, random_state=42)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(training_x, training_y)


KNeighborsClassifier(n_neighbors=3)

In [6]:
y_pred = knn.predict(testing_x)

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
report1 = classification_report(testing_y, y_pred)
print(report1)

              precision    recall  f1-score   support

           0       0.96      0.99      0.97     69080
           1       0.95      0.85      0.90     20955

    accuracy                           0.96     90035
   macro avg       0.95      0.92      0.93     90035
weighted avg       0.95      0.96      0.95     90035



In [ ]:
import urllib.parse
import ipaddress

feature_names = ['url_length', 'domain_length', 'has_https','parameter_count',
     'letter_count', 'nb_count', 'non_alpha_count', 'ques', 'slashes',
     'dashes', 'underscores', 'dots', 'has_php', 'has_html']

def count_symbols(url: str) -> tuple:
    letters = 0
    numbers = 0
    non_alpha_numeric = 0
    for char in url:
        if char.isalpha():
            letters += 1
        elif char.isdigit():
            numbers += 1
        else:
            non_alpha_numeric += 1
    return letters, numbers, non_alpha_numeric

def count_parameters(url: urllib.parse.ParseResult):
    parameters = url.query.split("&")
    return len(parameters)

def count_chars(url: str) -> tuple:
    ques = 0
    slashes = 0
    dashes = 0
    underscores = 0
    dots = 0
    for char in url:
        if char == "?":
            ques += 1
        elif char == "/":
            slashes += 1
        elif char == ".":
            dots += 1
        elif char == "-":
            dashes += 1
        elif char == "_":
            underscores += 1
    return ques, slashes, dashes, underscores, dots

def get_attributes(url: str):
    parsed_url = urllib.parse.urlparse(url)
    new_data = pd.DataFrame()
    url_length = len(url)
    domain_length = len(parsed_url.netloc)

    letter_count, nb_count, non_alpha_count = count_symbols(url)
    ques, slashes, dashes, underscores, dots = count_chars(url)

    has_php = 1 if "php" in url else 0
    has_html = 1 if "html" in url else 0
    has_https = 1 if "https" in url else 0
    parameter_count = count_parameters(parsed_url)

    return [url_length, domain_length, has_https,parameter_count,
            letter_count, nb_count, non_alpha_count, ques, slashes,
            dashes, underscores, dots, has_php, has_html]



In [ ]:
url = "https://www.google.com/"
url_attrs = pd.DataFrame([get_attributes(url)], columns=feature_names)

print(url_attrs)

   url_length  domain_length  parameter_count  letter_count  nb_count  \
0          23             14                1            17         0   

   non_alpha_count  ques  slashes  dashes  underscores  dots  has_php  \
0                6     0        3       0            0     2        0   

   has_html  
0         0  


In [ ]:
prediction = knn.predict(url_attrs)

print("prediction: ", prediction)

prediction:  [0]


In [11]:
import pickle

with open('sample_data/knn_model.sav', 'wb') as f:
    pickle.dump(knn, f)

**Part 2: Random Forest Model**


In [8]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=0)

classifier.fit(training_x, training_y)

RandomForestClassifier(max_depth=10, n_estimators=200, random_state=0)

In [9]:
y_pred2 = classifier.predict(testing_x)
report2 = classification_report(testing_y, y_pred2)
print(report2)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     69080
           1       1.00      0.95      0.97     20955

    accuracy                           0.99     90035
   macro avg       0.99      0.98      0.98     90035
weighted avg       0.99      0.99      0.99     90035



In [10]:
import pickle

with open('sample_data/rf_model.sav', 'wb') as f:
  pickle.dump(classifier, f)